In [ ]:
import data_analysis.data_processing as dp
import data_analysis.data_learning as dl

# reload module to bypass caching
import importlib
importlib.reload(dp)
importlib.reload(dl)

# treat it as a long path to avoid path length issues
test_window_path_home = dp.Path(r'\\?\C:\Users\jannis\Documents\HSP_IDS\Material\Aktuell\2025-02-17_11-14-33_192.168.1.0-normal_1\1554220324.748197-1554220354.748197')
test_window_path_remote = dp.Path(r'/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-04_00-03-20_192.168.1.0-normal_DDoS_1/1556203726.876922-1556203756.876922')

attack_data_set_path = dp.Path(r'/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff')

thirty_second_windows_paths = dp.getThirtySecondWindows(attack_data_set_path)
sample = thirty_second_windows_paths.groupby("type").sample(n=10, random_state=42).reset_index(drop=True)
print(sample.iloc[0][0])
thirty_second_windows_pcas = []

# for path in thirty_second_windows_paths["path"]:
#     tsw = dp.ThirtySecWindow(path)
#     thirty_second_windows_pcas.append(dl.create_feature_vector(tsw))












/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/2
/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/0
/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/2
/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/5
/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/-
/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/0
/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-08_15-56-59_192.168.1.0-normal_DoS_1/3
/home/hsp252/nas_mount/hunt